# dim_type_novelty

### Importación de librerías

In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

### Conexión a base y bodega de datos

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:
type_novelty = pd.read_sql_table('mensajeria_tiponovedad', engineSource)
type_novelty.rename(columns={
  "id": "key_dim_type_novelty", 
  "nombre": "name"
  }, inplace=True)

type_novelty

### Carga de datos

In [ ]:
type_novelty.to_sql("dim_type_novelty", engineDestination, index=False, if_exists="replace")